# Анализ данных сервиса для чтения книг по подписке

Цель проекта: проанализировать базу данных сервиса, чтобы сформулировать ценностное предложение для нового продукта.

План: 


    Выгрузить данные;
    Посчитать, сколько книг вышло после 1 января 2000 года;
    Для каждой книги посчитать количество обзоров и среднюю оценку;
    Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
    Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
    Посчитайть среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

**Шаг1. Подключение к базе данных.**

In [12]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa
# устанавливаем параметры
db_config = {
'user': '...', # имя пользователя
'pwd': '...', # пароль
'host': '...',
'port': ..., # порт подключения
'db': '...' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

Рассмотрю первую базу данных book.

In [13]:
query = '''SELECT * 
        FROM books 
        LIMIT 5'''
get_sql_data(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


База данных books содержит информацию об айди книги(book_id), айди автора(author_id), название книги(title), количество страниц(num_pages), дата публикации(publication_date), айди издательства(publisher_id).

In [14]:
query = """SELECT *
           FROM authors
           LIMIT 5;
        """

# выводим результат
get_sql_data(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Датасет authors содержит информацию об айди автора(author_id) и имя автора(author). 

In [15]:
query = """SELECT *
           FROM ratings
           LIMIT 5;
        """

# выводим результат
get_sql_data(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


База данных ratings содержит информацию об айди оценки(rating_id), айди книги(book_id), имя пользователя, оставившего оценку(username), оценка книги(rating).

In [16]:
query = """SELECT *
           FROM reviews
           LIMIT 5;
        """

# выводим результат
get_sql_data(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


База данных reviews содержит информацию об идентификатор обзора(review_id), идентификатор книги(book_id),  имя пользователя, написавшего обзор(username) и текст обзора(text).

In [17]:
query = """SELECT *
           FROM publishers
           LIMIT 5;
        """

# выводим результат
get_sql_data(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


База данных содержит идентификатор издательства(publisher_id) и  название издательства(publisher).

**Шаг2. Анализ данных.**

In [18]:
query = """SELECT COUNT(book_id)
           FROM books
           WHERE publication_date > '2000-01-01';
        """

# выводим результат
get_sql_data(query)

,count
0,819


Количество книг выпущенных после 01 января 2000 равно 819.

Посчитаю количество обзоров и среднюю оценку каждой книги. Выведеи первые 5 строк.

In [16]:
query = """SELECT r.book_title,
                  r.num_reviews,
                  r.average_rating
                  FROM (SELECT b.title AS book_title,
                 b.book_id as id,
       COUNT(DISTINCT r.review_id) AS num_reviews,
       COALESCE(AVG(rt.rating), 0) AS average_rating
       FROM books b
       LEFT JOIN reviews r ON b.book_id = r.book_id
       LEFT JOIN ratings rt ON b.book_id = rt.book_id
       GROUP BY id
       ORDER BY num_reviews DESC) AS r
       
 ;
        """

# выводим результат
get_sql_data(query)

,book_title,num_reviews,average_rating
0,Twilight (Twilight #1),7,3.662500
1,Water for Elephants,6,3.977273
2,The Glass Castle,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,The Natural Way to Draw,0,3.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,Essential Tales and Poems,0,4.000000


Мы видим что книги с большим количеством отзывов не гарантируют высокую оценку, так как книги в топ 5 по рейтингу не превышают оценку 4.2 балла. Максимальное количество обзоров на книге-7, такая книга одна и далее идут книги с 6 и менее количеством отзывов.

Определим издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [20]:
query = """SELECT p.publisher AS publisher_name,
                  COUNT(b.book_id) AS book_count
        FROM books b
        JOIN publishers p ON b.publisher_id = p.publisher_id
        WHERE b.num_pages > 50
        GROUP BY p.publisher
        ORDER BY book_count DESC
        LIMIT 1;
        """

# выводим результат
get_sql_data(query)

,publisher_name,book_count
0,Penguin Books,42


В базе данных издательство с максимальныи количеством выпущенных книг это Penguin Books.

Определим автора с самой высокой средней оценкой книг — учитывать будем только книги с 50 и более оценками.

In [21]:
query = """SELECT authors.author AS author_name,
                  round(AVG(r.average_rating), 2) AS average_rate    
           FROM books
           LEFT JOIN (SELECT book_id,
                          COUNT(rating) AS count,
                          AVG(rating) AS average_rating
                   FROM ratings
                   GROUP BY book_id) AS r
                ON r.book_id = books.book_id
           LEFT JOIN authors ON authors.author_id = books.author_id
           WHERE r.count >= 50
           GROUP BY author_name
           ORDER BY average_rate DESC
           LIMIT 1
        """

# выводим результат
get_sql_data(query)

,author_name,average_rate
0,J.K. Rowling/Mary GrandPré,4.28


Наибольшая средняя оценка у автора J.K. Rowling.

Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [22]:
query = """SELECT AVG(r.review_count) AS avg_review_count
           FROM ( SELECT username, COUNT(review_id) AS review_count
                  FROM reviews
                  GROUP BY username) r
           JOIN (SELECT username, COUNT(*) AS rating_count
                 FROM ratings
                 GROUP BY username
                 HAVING COUNT(*) > 48) rr ON r.username = rr.username;

   """
# выводим результат
get_sql_data(query)

,avg_review_count
0,24.0


Среднее количество обзоров на пользователя - 24.

**Вывод**


Книг выпущенных после 2000 - 819. Книги с большим количеством отзывов не гарантируют высокую оценку, так как книги в топ 5 по рейтингу не превышают оценку 4.2 балла.Самый рейтинговый автор - J.K. Rowling. В базе данных издательство с максимальныи количеством выпущенных книг это Penguin Books.Среднее количество обзоров на пользователя - 24.